# Sony Video Camera Data: Segmentation, Image Extraction, and Preparation

In [11]:
## Required python libraries
import pandas as pd
import numpy as np
import scipy as sp
import scipy.signal
import os
import math
import sys
import shutil
from scipy.interpolate import griddata
import subprocess
# to make this notebook's output stable across runs
np.random.seed(42)

pd.plotting.register_matplotlib_converters()


import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline
import seaborn as sns
import librosa
import matplotlib.font_manager as font_manager

#---------------opencv------------------------
import cv2
from IPython.display import Image, display
from tqdm import tqdm

In [2]:
from moviepy.editor import VideoFileClip
import imageio_ffmpeg as ffmpeg

In [3]:
# Where to save the figures, and dataset locations
PROJECT_ROOT_DIR = "../"

Multimodal_dataset_PATH = "/home/chenlequn/pan1/Dataset/LDED_acoustic_visual_monitoring_dataset"

Video_path = os.path.join(Multimodal_dataset_PATH, f'Sony Camera')
IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "result_images", 'Sony-Camera-Image')

os.makedirs(IMAGE_PATH, exist_ok=True)

## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [4]:
# Define function to save figures
def save_frames(frame, sample_index, counter, image_output_folder, fig_extension="png"):
    filename = f'sample_{sample_index}_{counter}.{fig_extension}'
    path = os.path.join(image_output_folder, filename)
    # print("Saving frame", filename)
    cv2.imwrite(path, frame)
    # cv2.imwrite(path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 100])


# Function to display video information
def display_video_info(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_duration = total_frames / fps
    # Convert total duration to minutes and seconds
    total_duration_min = int(total_duration // 60)
    total_duration_sec = int(total_duration % 60)

    print(f"Video FPS: {fps}")
    print(f"Total Frames: {total_frames}")
    print(f"Total Duration (seconds): {total_duration}")
    print(f"Total Duration: {total_duration_min} min {total_duration_sec} seconds")


    cap.release()

# Function to crop video by time and save frames
def crop_video_and_save_frames(video_path, image_output_folder, start_time, end_time, sample_index, target_fps=25):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    frame_interval = int(fps / target_fps)
    
    # Calculate total frames to process and initialize tqdm
    total_frames_to_process = (end_frame - start_frame) // frame_interval
    pbar = tqdm(total=total_frames_to_process, desc="Processing frames")
    
    counter = 1  # Initialize frame counter
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    while True:
        ret, frame = cap.read()
        if not ret or cap.get(cv2.CAP_PROP_POS_FRAMES) > end_frame:
            break  # Stop if the end of the segment is reached or on read error
        
        if (counter - 1) % frame_interval == 0:
            save_frames(frame, sample_index, counter, image_output_folder)
            pbar.update(1)
        
        counter += 1

    pbar.close()
    cap.release()


def format_time(seconds):
    """Converts time in seconds to HH:MM:SS format."""
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

def crop_video_and_save_frames_ffmpeg(video_path, image_output_folder, start_time, end_time, sample_index, target_fps=25):
    # Convert start_time and end_time to HH:MM:SS format
    start_timestamp = format_time(start_time)
    duration = end_time - start_time
    duration_timestamp = format_time(duration)
    total_frames = int(duration * target_fps)

    # Ensure output folder exists
    if not os.path.exists(image_output_folder):
        os.makedirs(image_output_folder)

    # Output pattern for frames
    output_pattern = os.path.join(image_output_folder, f"sample_{sample_index}_%d.png")

    # Build the FFmpeg command
    command = [
        'ffmpeg',
        '-ss', start_timestamp,                 # Start time
        '-t', duration_timestamp,               # Duration to process
        '-i', video_path,                       # Input file path
        '-vf', f'yadif,fps={target_fps}',       # Video filters
        '-q:v', '1',                            # Output quality (lower is better)
        '-start_number', '1',                   # Start numbering frames at 0
        '-progress', 'pipe:1',                  # Output progress to pipe
        output_pattern
    ]

    # Start the FFmpeg process and include a progress bar
    subprocess.run(command, check=True)

## Segment MTS video file into Images

In [29]:
sample_index = 23

# ----------------Define input video file path---------------
raw_video_file = os.path.join(Video_path, f'{sample_index}.MTS') 

# Output folders
image_output_folder = os.path.join(Multimodal_dataset_PATH, '25Hz',str(sample_index), 'sony-images')
sony_camera_image_folder = os.path.join(Multimodal_dataset_PATH, '25Hz',str(sample_index), 'sony_camera_images')

# Create output folders if they don't exist
if not os.path.exists(image_output_folder):
    os.makedirs(image_output_folder)

# Create output folders if they don't exist
if not os.path.exists(sony_camera_image_folder):
    os.makedirs(sony_camera_image_folder)

# Display video information
display_video_info(raw_video_file)

Video FPS: 25.0
Total Frames: 13728
Total Duration (seconds): 549.12
Total Duration: 9 min 9 seconds


In [23]:
# Define start and end times for cropping
# Sample 21: 0 - 182 seconds, in total, 4419 images (initial several layers are missing in the sony videos)
# Sample 22: 0 - 179 seconds, in total 
# Sample 23: 0 - 6 min 47 seconds = 407

# Sample 26: 0 - 6 min 43 seconds = 403
start_time = 0  # Start time in seconds 
end_time = 407   # End time in seconds

# Crop the video and save frames at 25 Hz
# crop_video_and_save_frames(raw_video_file, image_output_folder, start_time, end_time, sample_index, target_fps=25)

crop_video_and_save_frames_ffmpeg(raw_video_file, image_output_folder, start_time, end_time, sample_index, target_fps=25)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


frame=67
fps=0.00
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=1560000
out_time_ms=1560000
out_time=00:00:01.560000
dup_frames=0
drop_frames=0
speed=3.08x
progress=continue


frame=139
fps=137.75
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=4440000
out_time_ms=4440000
out_time=00:00:04.440000
dup_frames=0
drop_frames=0
speed= 4.4x
progress=continue


frame=205
fps=135.81
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=7080000
out_time_ms=7080000
out_time=00:00:07.080000
dup_frames=0
drop_frames=0
speed=4.69x
progress=continue


frame=274
fps=136.15
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=9840000
out_time_ms=9840000
out_time=00:00:09.840000
dup_frames=0
drop_frames=0
speed=4.89x
progress=continue


...

frame=10072
fps=73.03
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=401600000
out_time_ms=401600000
out_time=00:06:41.600000
dup_frames=0
drop_frames=0
speed=2.91x
progress=continue


frame=10126
fps=73.15
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=403760000
out_time_ms=403760000
out_time=00:06:43.760000
dup_frames=0
drop_frames=0
speed=2.92x
progress=continue


frame=10155
fps=73.10
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=404920000
out_time_ms=404920000
out_time=00:06:44.920000
dup_frames=0
drop_frames=0
speed=2.91x
progress=continue


frame=10175
fps=72.97
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=405720000
out_time_ms=405720000
out_time=00:06:45.720000
dup_frames=0
drop_frames=0
speed=2.91x
progress=continue
frame=10175
fps=72.50
stream_0_0_q=-0.0
bitrate=N/A
total_size=N/A
out_time_us=407000000
out_time_ms=407000000
out_time=00:06:47.000000
dup_frames=0
drop_frames=0
speed= 2.9x
progress=end


frame=10175 fps= 72 q=-0.0 Lsize=N/A time=00:06:47.00 bitrate=N/A speed= 2.9x    
video:18851864kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown


## Fix the index to match with the coaxial image stream.

In [30]:
import os

# Calculate the index offset
index_offset = 11155 - 10103
# Sample 21: offset is 5030 (coaxial) - 4430
# Sample 22: offset is 4623 (coaxial) - 4469
# Sample 23: offset is 11154 (coaxial) - 10090

# Sample 26: offset is 10749 (coaxial) - 10010

# Get all PNG files in the directory
image_files = [f for f in os.listdir(image_output_folder) if f.endswith(".png")]

# Loop through all files in the directory with a progress bar
for filename in tqdm(image_files, desc="Renaming images"):
    # the format 'sample_<sample_index>_<current_index>.png'
    parts = filename.split('_')
    if len(parts) == 3:
        current_index_part = parts[2].split('.')[0]  # This gets the numeric part of '<current_index>.png'
        try:
            current_index = int(current_index_part)
            # Calculate the new index
            new_index = current_index + index_offset
            # Generate the new filename
            new_filename = f'sample_{sample_index}_{new_index}.png'
            # Full path for current and new file names
            current_file_path = os.path.join(image_output_folder, filename)
            new_file_path = os.path.join(sony_camera_image_folder, new_filename)
            # Copy the file to the new folder with the new name
            shutil.copy2(current_file_path, new_file_path)
        except ValueError:
            # In case the filename format is unexpected and doesn't contain an index
            continue  # Skip files that do not match the expected format

Renaming images: 100%|███████████████████████████████████████████████████████████████████████| 10175/10175 [02:56<00:00, 57.68it/s]
